# TfL cycling usage data 2021 - 2023 Analytics

In [ ]:
# Import necessary libraries
import os
import pandas as pd
from dateutil import parser
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Load all input data from TFL cycling usage data files

In [ ]:
folder_path = "tfl_csv_files"

# List all CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Initialize an empty list to store DataFrames
dataframes = []
i = 0
print(f"Looking for CSV files in: {os.path.abspath(folder_path)}")

# --- Define the mapping from variant names to standard names ---
column_mapping = {
    "Start date": "Start Date",           # Standardize 'Start Date'
    "End date": "End Date",             # Standardize 'End Date'
    "Start station": "StartStation Name", # Standardize 'Start Station Name'
    "End station": "EndStation Name",   # Standardize 'End Station Name'
    "Start station number": "StartStation Id", # Map number to Id
    "End station number": "EndStation Id",   # Map number to Id
    "Bike number": "Bike Id",             # Map number to Id
    "Number": "Rental Id"                 # Map 'Number' to 'Rental Id'
}

if not csv_files:
    print("No CSV files found in the specified folder.")
else:
    print(f"Found {len(csv_files)} CSV files. Loading and Standardizing...")
    for file in csv_files:
        file_path = os.path.join(folder_path, file)
        print(f"Attempting to load file {i+1}: {file}...")
        try:
            df = pd.read_csv(file_path, engine='python') 

            # --- Standardize column names for the current DataFrame ---
            rename_dict_for_this_df = {
                variant_name: standard_name
                for variant_name, standard_name in column_mapping.items()
                if variant_name in df.columns
            }

            # Apply the renaming if any columns matched the mapping
            if rename_dict_for_this_df:
                df = df.rename(columns=rename_dict_for_this_df)
                print(f"   Renamed columns in {file}: {list(rename_dict_for_this_df.keys())} -> {list(rename_dict_for_this_df.values())}")

            dataframes.append(df)
            i += 1
            print(f"Successfully loaded and processed: {file} with shape {df.shape}")

        except Exception as e:
            # Print the error and the file that caused it
            print(f"-----------------------------------------------------")
            print(f"Error loading file {file}: {e}")
            print(f"Skipping this file.")
            print(f"-----------------------------------------------------")
            # continue # Or break if you want to stop on error

    # Check if any dataframes were loaded successfully
    if dataframes:
        print("\nConcatenating DataFrames...")
        combined_df1 = pd.concat(dataframes, axis=0, ignore_index=True, sort=False)

        # Display the combined DataFrame info
        print("\nCombined DataFrame Info:")
        print(f"Shape: {combined_df1.shape}")

        # --- Check NaN percentages again ---
        print("\nPercentage of NaN values per column (after standardization):")
        nan_percentage = (combined_df1.isna().sum() / len(combined_df1)) * 100
        print(nan_percentage)
    else:
        print("No dataframes were successfully loaded to concatenate.")

#### Standardizing column names in order to merge multiple dataframes

In [ ]:
# --- Define the mapping from variant names to standard names ---
column_mapping = {
    "Start date": "Start Date",           # Standardize 'Start Date'
    "End date": "End Date",             # Standardize 'End Date'
    "Start station": "StartStation Name", # Standardize 'Start Station Name'
    "End station": "EndStation Name",   # Standardize 'End Station Name'
    "Start station number": "StartStation Id", # Map number to Id
    "End station number": "EndStation Id",   # Map number to Id
    "Bike number": "Bike Id",             # Map number to Id
    "Number": "Rental Id"                 # Map 'Number' to 'Rental Id'
}

folder_path = "part2"
# List all CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Initialize an empty list to store DataFrames
dataframes = []
i = 0
print(f"Looking for CSV files in: {os.path.abspath(folder_path)}")
if not csv_files:
    print("No CSV files found in the specified folder.")
else:
    print(f"Found {len(csv_files)} CSV files. Loading and Standardizing...")
    for file in csv_files:
        file_path = os.path.join(folder_path, file)
        print(f"Attempting to load file {i+1}: {file}...")
        try:
            df = pd.read_csv(file_path, engine='python')

            # --- Standardize column names for the current DataFrame ---
            rename_dict_for_this_df = {
                variant_name: standard_name
                for variant_name, standard_name in column_mapping.items()
                if variant_name in df.columns
            }

            # Apply the renaming if any columns matched the mapping
            if rename_dict_for_this_df:
                df = df.rename(columns=rename_dict_for_this_df)
                print(f"   Renamed columns in {file}: {list(rename_dict_for_this_df.keys())} -> {list(rename_dict_for_this_df.values())}")

            dataframes.append(df)
            i += 1
            print(f"Successfully loaded and processed: {file} with shape {df.shape}")

        except Exception as e:
            # Print the error and the file that caused it
            print(f"-----------------------------------------------------")
            print(f"Error loading file {file}: {e}")
            print(f"Skipping this file.")
            print(f"-----------------------------------------------------")
            # continue # Or break if you want to stop on error

    # Check if any dataframes were loaded successfully
    if dataframes:
        print("\nConcatenating DataFrames...")
        combined_df2 = pd.concat(dataframes, axis=0, ignore_index=True, sort=False)

        # Display the combined DataFrame info
        print("\nCombined DataFrame Info:")
        print(f"Shape: {combined_df2.shape}")

        # --- Check NaN percentages again ---
        print("\nPercentage of NaN values per column (after standardization):")
        nan_percentage = (combined_df2.isna().sum() / len(combined_df2)) * 100
        print(nan_percentage)
    else:
        print("No dataframes were successfully loaded to concatenate.")

In [ ]:
combined_df = pd.concat([combined_df1, combined_df2], axis=0, ignore_index=True)
combined_df.shape

#### Parsing start date and end date since they are in different format in various csv files

In [ ]:
df = combined_df[["Bike Id", "Start Date", "End Date", "Rental Id", "StartStation Id", "StartStation Name", "EndStation Id", "EndStation Name"]]

# Step 1: Parsing with pandas first (handles most common formats)
df['Parsed Start Date'] = pd.to_datetime(df['Start Date'], errors='coerce')

# Step 2: Identifying rows that failed to parse
mask_failed = df['Parsed Start Date'].isna() & df['Start Date'].notna()

# Step 3: Trying dateutil.parser.parse on the failed ones
def robust_parse(date_str):
    try:
        return parser.parse(date_str)
    except Exception:
        return pd.NaT

# Apply only to rows that failed in first attempt
df.loc[mask_failed, 'Parsed Start Date'] = df.loc[mask_failed, 'Start Date'].apply(robust_parse)

# Optional: Drop or log rows that still couldn't be parsed
still_failed = df['Parsed Start Date'].isna() & df['Start Date'].notna()
if still_failed.any():
    print(f"Warning: {still_failed.sum()} dates could not be parsed.")

In [ ]:
# Step 1: Parsing with pandas first (handles most common formats)
df['Parsed End Date'] = pd.to_datetime(df['End Date'], errors='coerce')

# Step 2: Identifying rows that failed to parse
mask_failed = df['Parsed End Date'].isna() & df['End Date'].notna()

# Step 3: Trying dateutil.parser.parse on the failed ones
def robust_parse(date_str):
    try:
        return parser.parse(date_str)
    except Exception:
        return pd.NaT

# Apply only to rows that failed in first attempt
df.loc[mask_failed, 'Parsed End Date'] = df.loc[mask_failed, 'End Date'].apply(robust_parse)

In [ ]:
# Calculate travel duration in minutes
df["duration_min"] = (df["Parsed End Date"] - df["Parsed Start Date"]).dt.total_seconds() / 60.0  # Duration in minutes 

In [ ]:
print(df["Parsed Start Date"].isnull().sum())
df.head()

In [ ]:
print(df.shape)
# Drop duplicates
df.drop_duplicates(inplace=True)
print(df.shape)

In [ ]:
# Check which columns have NaN values
df.isna().sum()

## Exploratory Data Analysis

In [ ]:
print("--- DataFrame Info ---")
# Display the structure and data types of the dataframe
df.info()
print("\n")

In [ ]:
# Get the top 3 start and end stations based on the number of rentals

top_3_start_stations = df['StartStation Name'].value_counts().head(3)
top_3_end_stations = df['EndStation Name'].value_counts().head(3)

print("--- Top Start Stations (up to 3) ---")
print(top_3_start_stations)
print("\n")

print("--- Top End Stations (up to 3) ---")
print(top_3_end_stations)
print("\n")

In [ ]:
rides_within_half_hour = df[df['duration_min'] <= 30].shape[0] 

# Count the total number of rides
total_rides = df.shape[0] 

# Calculate the percentage
percentage_within_half_hour = (rides_within_half_hour / total_rides) * 100
percentage_within_half_hour

In [ ]:
# --- Visualizing Distribution of Rental duration Data using Plotly ---

print("--- Generating Plots (using Plotly) ---")
filtered_df = df[(df['duration_min'] > 0) & (df['duration_min'] <= 60)].copy()

fig_hist = px.histogram(filtered_df, x='duration_min',
                        title='Distribution of Rental Durations',
                        labels={'duration_min': 'Duration (minutes)'},
                        nbins=6
                       )
fig_hist.update_layout(bargap=0.1) 
fig_hist.show()

In [ ]:
fig_stations = make_subplots(rows=1, cols=2, subplot_titles=('Top Start Stations', 'Top End Stations'))

# Top Start Stations Bar Chart
fig_stations.add_trace(
    go.Bar(x=top_3_start_stations.index, y=top_3_start_stations.values, name='Start Stations', marker_color='green'),
    row=1, col=1
)

# Top End Stations Bar Chart
fig_stations.add_trace(
    go.Bar(x=top_3_end_stations.index, y=top_3_end_stations.values, name='End Stations', marker_color='orange'),
    row=1, col=2
)

# Update layout
fig_stations.update_layout(title_text='Top Start and End Station Analysis', showlegend=False)
fig_stations.update_xaxes(title_text="Station Name", row=1, col=1)
fig_stations.update_xaxes(title_text="Station Name", row=1, col=2)
fig_stations.update_yaxes(title_text="Frequency", row=1, col=1)
fig_stations.update_yaxes(title_text="Frequency", row=1, col=2)
fig_stations.show()

In [ ]:
# Feature Engineering & Time-Based Plots
df['Start Hour'] = df['Parsed Start Date'].dt.hour
df['Start DayOfWeek'] = df['Parsed Start Date'].dt.day_name()

# Rentals by Hour of Day
hourly_counts = df['Start Hour'].value_counts().sort_index()
fig_hour = px.bar(x=hourly_counts.index, y=hourly_counts.values,
                  title='Number of Rentals by Hour of Day',
                  labels={'x': 'Hour of Day', 'y': 'Number of Rentals'})
fig_hour.update_layout(xaxis = dict(tickmode = 'linear', dtick = 1))
fig_hour.show()

In [ ]:
# Rentals by Day of Week
daily_counts = df['Start DayOfWeek'].value_counts()

days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daily_counts = daily_counts.reindex(days_order).fillna(0)

fig_day = px.bar(x=daily_counts.index, y=daily_counts.values,
                 title='Number of Rentals by Day of Week',
                 labels={'x': 'Day of Week', 'y': 'Number of Rentals'})
fig_day.show()

In [ ]:
print("--- Performing Monthly and Yearly Distribution Analysis ---")

# 1. Feature Engineering: Extract Month and Year
# Check if columns already exist to avoid errors on re-runs
if 'Start Month Name' not in df.columns:
    df['Start Month Name'] = df['Parsed Start Date'].dt.month_name()
if 'Start Year' not in df.columns:
    df['Start Year'] = df['Parsed Start Date'].dt.year

# 2. Calculate Yearly Distribution
yearly_counts = df['Start Year'].value_counts().sort_index()
print("\n--- Yearly Rental Counts ---")
print(yearly_counts)

# 3. Calculate Monthly Distribution
monthly_counts = df['Start Month Name'].value_counts()

# Define the correct order for months
month_order = ['January', 'February', 'March', 'April', 'May', 'June',
               'July', 'August', 'September', 'October', 'November', 'December']

# Reindex the monthly counts to ensure chronological order and fill missing months with 0
monthly_counts = monthly_counts.reindex(month_order, fill_value=0)

print("\n--- Monthly Rental Counts (Ordered) ---")
print(monthly_counts)

# 4. Plot Yearly Distribution
fig_yearly = px.bar(x=yearly_counts.index, y=yearly_counts.values,
                    title='Yearly Distribution of Rentals',
                    labels={'x': 'Year', 'y': 'Number of Rentals'},
                    text_auto=True) 
fig_yearly.update_layout(xaxis_type='category')
print("\n--- Showing Yearly Distribution Plot ---")
fig_yearly.show()

In [ ]:
# 5. Plot Monthly Distribution
fig_monthly = px.bar(x=monthly_counts.index, y=monthly_counts.values,
                     title='Monthly Distribution of Rentals',
                     labels={'x': 'Month', 'y': 'Number of Rentals'},
                     text_auto=True)
print("\n--- Showing Monthly Distribution Plot ---")
fig_monthly.show()

In [ ]:
columns_to_drop = ['Start Date', 'End Date']
df1 = df.drop(columns=columns_to_drop)
df1.head(2)

In [ ]:
df1.shape

In [ ]:
# Sample for understanding the underlying usage distribution

a = df[df['StartStation Name'] == 'Hyde Park Corner, Hyde Park']
routes = a.groupby(['StartStation Name', 'EndStation Name']).size().reset_index(name='Count')
# Sorts the routes by count in descending order to find the most frequent
most_frequent_routes = routes.sort_values(by='Count', ascending=False).head(5) #to get top 5 routes
most_frequent_routes

## Data Analytics Usecase 1 - Station Analysis

In [ ]:
# --- Station Analysis ---

# 1. Identify the most popular start stations
#    Counts the occurrences of each unique start station name
popular_start_stations = df['StartStation Name'].value_counts().head() 
print("Most Popular Start Stations:")
print(popular_start_stations)
print("\n") 

# 2. Identify the most popular end stations
#    Counts the occurrences of each unique end station name
popular_end_stations = df['EndStation Name'].value_counts().head() 
print("Most Popular End Stations:")
print(popular_end_stations)
print("\n") 

# 3. Identify the most frequently used routes
routes = df.groupby(['StartStation Name', 'EndStation Name']).agg(
    Count=('Rental Id', 'size'),             # Calculate trip count for the route
    MinDuration=('duration_min', 'min'),     # Calculate minimum duration
    MedianDuration=('duration_min', 'median'), # Calculate median duration
    MaxDuration=('duration_min', 'max')      # Calculate maximum duration
).reset_index() # Convert the grouped result back to a DataFrame

# Sort the routes by count in descending order to find the most frequent
most_frequent_routes = routes.sort_values(by='Count', ascending=False).head(20) 
most_frequent_routes['MinDuration'] = most_frequent_routes['MinDuration'].clip(lower=0)
print("Most Frequent Routes with Duration Statistics (Min, Median, Max):")

most_frequent_routes

## Data Analytics Usecase 2 - Bike stock balancing Strategy (Demand-Supply matching)

In [ ]:
# --- Bike stock balancing Strategy (Simplified Approach) ---

# 1. Calculate Historical Demand (Starts) and Supply (Ends) per Station
#    This uses past data as a proxy for current/near-future imbalance.
start_counts = df['StartStation Name'].value_counts()
end_counts = df['EndStation Name'].value_counts()

# 2. Combine into a single DataFrame showing net flow for each station
station_flow = pd.DataFrame({
    'Demand (Starts)': start_counts,
    'Supply (Ends)': end_counts
}).fillna(0) # Fill stations that only appear as start or end with 0

# Calculate Net Demand: Positive means more bikes leave than arrive (deficit potential)
#                        Negative means more bikes arrive than leave (surplus potential)
station_flow['Net Demand'] = station_flow['Demand (Starts)'] - station_flow['Supply (Ends)']

# 3. Identify Stations with Potential Surplus and Deficit
#    Surplus stations have more bikes arriving than leaving (Net Demand < 0)
#    Deficit stations have more bikes leaving than arriving (Net Demand > 0)
surplus_stations = station_flow[station_flow['Net Demand'] < 0].sort_values('Net Demand')
deficit_stations = station_flow[station_flow['Net Demand'] > 0].sort_values('Net Demand', ascending=False)

print("--- Station Flow Analysis ---")
print(station_flow.head())
print("\nPotential Surplus Stations (More bikes arriving):")
print(surplus_stations.head())
print("\nPotential Deficit Stations (More bikes leaving):")
print(deficit_stations.head())

# 4. Suggest Rebalancing Moves (Basic Pairing)
#    This simple example pairs stations with the biggest surplus to those with the biggest deficit.

print("\n--- Suggested Rebalancing Moves (Simplified) ---")
# Convert surplus/deficit stations to iterables for pairing
surplus_list = list(surplus_stations.index)
deficit_list = list(deficit_stations.index)
num_moves = min(len(surplus_list), len(deficit_list)) # Max possible pairs

if not surplus_list or not deficit_list:
    print("No rebalancing needed based on historical net demand, or data is insufficient.")
else:
    print("Move bikes FROM (Surplus) -> TO (Deficit):")
    # Simple pairing: match largest surplus with largest deficit, second largest, etc.
    #for i in range(num_moves):
    for i in range(5):
        from_station = surplus_list[i]
        to_station = deficit_list[i]
        # Calculate the theoretical number of bikes to move (limited by supply/demand)
        bikes_to_move = min(abs(surplus_stations.loc[from_station, 'Net Demand']),
                            deficit_stations.loc[to_station, 'Net Demand'])
        print(f"- Move {int(bikes_to_move)} bikes from '{from_station}' to '{to_station}'")

    print("\nNote: This is a simplified suggestion based on overall historical flow.")